In [5]:
import pandas as pd
import matplotlib.pyplot as plt

# Box 2D + Mountain Car
dfbu = pd.read_csv("keep_csv/box2d_all.csv")[['post', 'config.algo', 'config.env_name', 'ureward_mean', 'ureward_std', 'mdim_mean', 'cdim_mean', 'mdim_nan_mean', 'mdim_std', 'nreward_mean', 'nreward_std', 'len_mean',
                             'cdim_nan_mean']]#, 'config.post']]
# Bullet envs
dfb2 = pd.read_csv("keep_csv/train_all.csv")[['post', 'config.algo', 'config.env_name', 'ureward_mean', 'ureward_std', 'mdim_mean', 'cdim_mean', 'mdim_nan_mean', 'mdim_std', 'nreward_mean', 'nreward_std', 'len_mean',
                             'cdim_nan_mean']]#, 'conf

df = pd.concat([dfbu, dfb2])

In [6]:
# Some sort of gross pandas manipulation to get close to the table format we want
df2 = pd.DataFrame()
env_disp_name = "Environment"
algo_disp_name = "Algorithm"
o_disp_name = "Reward Baseline"
a_disp_name = "Reward W/ Training"

df2[env_disp_name] = df[df['post'] == 'iden']['config.env_name']
df2[algo_disp_name] = df[df['post'] == 'iden']['config.algo']
df2['brew_mean'] = df[df['post'] == 'iden']['ureward_mean']
df2['brew_std'] = df[df['post'] == 'iden']['ureward_std']
df2['arew_mean'] = df[df['post'] == 'postprocess_default']['ureward_mean'].to_numpy()
df2['arew_std'] = df[df['post'] == 'postprocess_default']['ureward_std'].to_numpy()

df2 = df2.round({'brew_mean':2,'brew_std':1,'arew_mean':2, "arew_std":1})
df2['brew_mean'] = df2['brew_mean'].astype(int)
df2['brew_mean'] = df2[['brew_mean', 'brew_std']].astype(str).agg(' ± '.join, axis=1) 
df2.drop('brew_std', axis=1, inplace=True)
df2['arew_mean'] = df2['arew_mean'].astype(int)
df2['arew_mean'] = df2[['arew_mean', 'arew_std']].astype(str).agg(' ± '.join, axis=1) 

df2.drop('arew_std', axis=1, inplace=True)
df2 = df2.rename({'brew_mean':'Baseline Return'}, axis=1)
df2 = df2.rename({'arew_mean':'Refined  Return'}, axis=1)

env_dict = {}
for env in df2['Environment']:
    env_dict[env] = 1 

df2 = df2.set_index(["Algorithm", "Environment"]).sort_index()
envs = list(env_dict.keys())

index_list = []
for algo in ["a2c","ppo","ddpg","td3","sac","tqc"]:
    for env in envs:
        index_list.append((algo,env))
        
df2 = df2.reindex(index_list)# ("ppo", envs), ("ddpg", envs), ("td3",envs), ("td3", envs) ,("tqc", envs)])
df2 = df2.reset_index()

#index = pd.MultiIndex.from_frame(df2)
#df2.set_index(["Algorithm","Environment"])
#df2.set_index(index).drop(["Environment", "Algorithm"],axis=1)

In [4]:
# Print as latex, we manually inserted this with some modifications in formatting to the manuscript
for env in env_dict.keys():
    dfp = df2[df2['Environment'] == env].drop(["Algorithm", "Environment"],axis=1,inplace=False).transpose()
    print(env)
    print(dfp.to_latex(multirow=True, header=False))

LunarLanderContinuous-v2
\begin{tabular}{lllllll}
\toprule
Baseline Return &  96 ± 148.3 &  277 ± 23.0 &   247 ± 92.5 &  199 ± 99.4 &  226 ± 103.5 &  280 ± 23.0 \\
Refined  Return &  208 ± 88.6 &  270 ± 22.3 &  217 ± 149.3 &  259 ± 21.7 &   279 ± 14.0 &  288 ± 17.7 \\
\bottomrule
\end{tabular}

BipedalWalker-v3
\begin{tabular}{lllllll}
\toprule
Baseline Return &  301 ± 1.0 &  207 ± 98.4 &  199 ± 140.0 &  318 ± 2.4 &  261 ± 116.9 &  292 ± 125.1 \\
Refined  Return &  313 ± 1.1 &   325 ± 1.0 &    291 ± 1.5 &  334 ± 0.6 &    321 ± 1.2 &    344 ± 0.2 \\
\bottomrule
\end{tabular}

BipedalWalkerHardcore-v3
\begin{tabular}{lllllll}
\toprule
Baseline Return &   78 ± 109.5 &  131 ± 123.3 &  NaN &  -94 ± 12.7 &  -34 ± 88.8 &  224 ± 101.7 \\
Refined  Return &  145 ± 100.9 &   131 ± 98.6 &  NaN &   -24 ± 5.3 &   18 ± 69.9 &  243 ± 100.1 \\
\bottomrule
\end{tabular}

Pendulum-v0
\begin{tabular}{lllllll}
\toprule
Baseline Return &  -230 ± 99.2 &  -171 ± 127.4 &  -157 ± 99.3 &  -110 ± 83.5 &  -116 ± 1

In [13]:
# Hack for visualizing, can only load one environment / process with rendering. Need to restart NB to change the env
model_dir = "./keep_agents/train_all"

import os
from seagul.zoo3_utils import load_zoo_agent, ALGOS, do_rollout_stable
import pybullet_envs
import gym

viz_env = "Walker2DBulletEnv-v0"
viz_algo = "ddpg"
model_dict = {}
env_dict = {}

for algo_file in os.scandir(model_dir):
    algo_dir = algo_file.path
    algo = algo_dir.split("/")[-1]
    if algo == viz_algo:
        model_dict[algo] = {}
        env_dict[algo] = {}
        for env_file in os.scandir(algo_dir):
            env_dir = env_file.path
            env_name = env_dir.split("/")[-1]
            model_dict[algo][env_name] = {}
            env_dict[algo][env_name] = {}
            if viz_env in env_name:

                env, original_model = load_zoo_agent(env_name, algo, render=False, env_kwargs={"render":True})
                env.close()
                env_dict[algo][env_name] = env
                model_dict[algo][env_name]['original'] = original_model
                for pkl_file in os.scandir(env_dir):
                    post_name = pkl_file.path.split("/")[-1].split(".")[0]
                    model_dict[algo][env_name][post_name] = ALGOS[algo].load(pkl_file.path, env=env, device='cpu')

/home/sgillen/work/external/rl-baselines3-zoo/rl-trained-agents/ddpg/Walker2DBulletEnv-v0_1/Walker2DBulletEnv-v0


/home/sgillen/anaconda3/envs/baselines/lib/python3.6/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


In [ ]:
model = model_dict[viz_algo][viz_env]['postprocess_default']
#env = gym.make(viz_env, render=True)
o,a,r,i = do_rollout_stable(env, model, render_wait=0.002, render=True, return_on_done=False, override_reset=False, num_steps=10000)
print(len(r))
print(i)